In [59]:
from __future__ import print_function, division
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import ElasticNetCV
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [2]:
import pickle
with open("combined_dummified_df.pkl", 'rb') as picklefile: 
    combined_df = pickle.load(picklefile)

In [4]:
# make datetime object into date
combined_df['Year'] = combined_df.BEGIN_YEARMONTH.apply(lambda x: x.year - 1955)
combined_df['Month'] = combined_df.BEGIN_YEARMONTH.apply(lambda x: x.month)
combined_df['DAMAGE_SCORE'] = combined_df['DAMAGE_PROPERTY'] + (8e6)*combined_df['DEATHS_DIRECT']

In [5]:
front = combined_df[['DAMAGE_SCORE', 'Year', 'Month']]
combined_df = combined_df.drop(columns = ['DAMAGE_SCORE', 'Year', 'Month'])
combined_df = combined_df.drop(columns = ['BEGIN_YEARMONTH', 'EVENT_ID'])
combined_df = pd.concat([front, combined_df], axis = 1)
combined_df = combined_df.drop(columns = ['INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT'])
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504723 entries, 0 to 504722
Data columns (total 87 columns):
DAMAGE_SCORE                                504723 non-null float64
Year                                        504723 non-null int64
Month                                       504723 non-null int64
Temperature                                 504723 non-null float64
Precipitation                               504723 non-null float64
MAGNITUDE                                   504723 non-null float64
BEGIN_LAT                                   504723 non-null float64
BEGIN_LON                                   504723 non-null float64
DAMAGE_PROPERTY                             504723 non-null float64
STATE[T.ALASKA]                             504723 non-null float64
STATE[T.AMERICAN SAMOA]                     504723 non-null float64
STATE[T.ARIZONA]                            504723 non-null float64
STATE[T.ARKANSAS]                           504723 non-null float64
STATE[T.A

In [6]:
# 9 million human life
def categorical_labeling(damage):
    if damage > 100000:
        return 'Major'
    elif damage > 1000:
        return 'Minor'
    else:
        return 'Negligible'

In [7]:
def binary_labeling(damage):
    if damage > 100000:
        return 1
    else:
        return 0

In [8]:
# make labels
category_list = []
combined_df['Category'] = combined_df.DAMAGE_SCORE.apply(categorical_labeling)
binary_list = []
combined_df['Binary'] = combined_df.DAMAGE_SCORE.apply(binary_labeling)

In [9]:
# set up data with stratified sampling
X = combined_df.drop(columns = ['Category','Binary', 'DAMAGE_SCORE', 'DAMAGE_PROPERTY'])
y = combined_df.Binary
y_cat = combined_df.Category
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)
# X_cattrain, X_cattest, y_cattrain, y_cattest = train_test_split(X, y_cat, test_size=0.2, stratify = y_cat)

In [30]:
X_train.shape

(403778, 85)

In [57]:
X_train_g = X_train.iloc[:,0:7]
X_test_g = X_test.iloc[:,0:7]
X_train_b = X_train.iloc[:,7:]
X_test_b = X_test.iloc[:,7:]

In [61]:
nbg = GaussianNB()
nbg.fit(X_train_g, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [62]:
nbb = BernoulliNB()
nbb.fit(X_train_b, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [64]:
nbg.predict_proba(X_test_g)
nbb.predict_proba(X_test_b)

array([[0.98791779, 0.01208221],
       [0.98651941, 0.01348059],
       [0.98406654, 0.01593346],
       ...,
       [0.99217713, 0.00782287],
       [0.97844782, 0.02155218],
       [0.99437781, 0.00562219]])

In [ ]:
# combine two naive bayes model

In [ ]:
from sklearn.externals import joblib
joblib.dump(grid, 'naivebayes_f1.joblib')